# Implement a feedback loop to update the model periodically with fresh manufacturing data.

In [2]:
# Import the reuire library
import numpy as np
import xgboost as xgb
from keras.applications import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
from sklearn.model_selection import train_test_split
import os
from tqdm import tqdm
import pickle
from sklearn.metrics import accuracy_score ,classification_report,confusion_matrix,roc_auc_score,precision_score,recall_score
from sklearn.metrics import f1_score,roc_curve,precision_recall_curve
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## DATA PREPROCESSING AND FEATURE ENGINERRING

In [3]:
# Load pre-trained ResNet-50 model
resnet_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

In [4]:
# Load and preprocess images 
# Here we will load the image with help of keras image preprocessing module and reshape it in 224x224 size
# And process it with the help of Resnet inbuild preprocessing  funtion
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [5]:
# Initiate the empty list to store image and label array
X = []
y = []

In [6]:
# Get the current directory
current_dir = os.getcwd()

# Get the parent directory (one level up)
current_dir = os.path.dirname(current_dir)


# Get the parent directory (one level up)
current_dir = os.path.dirname(current_dir)

# Get the parent directory (one level up)
parent_dir = os.path.dirname(current_dir)

# Print the parent directory
print("Parent Directory:", parent_dir)

Parent Directory: E:\upgrade_capston_project-main


In [7]:
# Dataset path
directory = parent_dir+"/datasets/feedbackLoopData/Digital images of defective and good condition tyres"

In [8]:
# Itertate through Dataset and its subfolder to process image and extract the feature
for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)
        for filename in tqdm(os.listdir(label_dir)):
            img_path = os.path.join(label_dir, filename)
            img_features = resnet_model.predict(preprocess_image(img_path))
            X.append(img_features)
            y.append(label)


  0%|                                                                                                                                                                                       | 0/4 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step


 25%|███████████████████████████████████████████▊                                                                                                                                   | 1/4 [00:11<00:33, 11.26s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 475ms/step


 50%|███████████████████████████████████████████████████████████████████████████████████████▌                                                                                       | 2/4 [00:12<00:10,  5.23s/it]

...

 50%|███████████████████████████████████████████████████████████████████████████████████████▌                                                                                       | 2/4 [00:01<00:01,  1.34it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 484ms/step


 75%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                           | 3/4 [00:02<00:00,  1.26it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 453ms/step


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:03<00:00,  1.32it/s]


In [9]:
# Convert to NumPy arrays
X = np.array(X)
y = np.array(y)

In [10]:
# Creating copy of feature
X1 = X.copy()

In [11]:
# Reshape the feature to make it compatible with Machine learning models 
X1 = np.reshape(X1 ,(X1.shape[0],X1.shape[1]*X1.shape[2]))

In [12]:
# Creating a copy of labels
y1 = y.copy()

In [13]:
# Replace all occurrences defective as 0 and good as 1
for i in range(len(y1)):
    if y1[i]=='defective' : 
        y1[i] = 0
    else:
        y1[i] = 1

In [14]:
# Conert it into string to int
y1=y1.astype(int)

In [26]:
# Split dataset into training and testing sets in 80-20 ratio
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.1, random_state=42)

In [27]:
model_dir = parent_dir+'/models/'
model_dir

'E:\\upgrade_capston_project-main/models/'

In [28]:
# Load the saved model from file
with open(os.path.join(model_dir,'RESNET50_xgbClassifier_model.pkl'), 'rb') as f:
    xgb_model = pickle.load(f)

# Trained the PreTrained Model with new dataset

In [29]:
model = xgb.XGBClassifier()
# Train the model
model = model.fit(X_train, y_train,xgb_model = xgb_model)

In [30]:
# Save the trained model to a file
with open(os.path.join(model_dir,'RESNET50_xgbClassifier_model.pkl'),'wb') as f:
    pickle.dump(model, f)

# I don't have much fresh dataset , so i just uploaded 4-4 picture in defective and good , so accuracy can't be calculaed in much efficient way due to lack of train and test data.So i tested with existing test dataset and its working upto the mark and commented the code.
# In Real world , We will be having suffiecient freash data to train and test.

In [41]:
#  preprocessed_data_dir = parent_dir+'/datasets/processed_dataset/'

In [42]:
# # Load X_test from file
# with open(os.path.join(preprocessed_data_dir,'resnetX_test.pkl'), 'rb') as f:
#     X_test = pickle.load(f)
    
# # Load y_test from file
# with open(os.path.join(preprocessed_data_dir,'resnety_test.pkl'), 'rb') as f:
#     y_test = pickle.load(f)

In [43]:
# # Replace all occurrences defective as 0 and good as 1
# for i in range(len(y_test)):
#     if y_test[i]=='defective' : 
#         y_test[i] = 0
#     else:
#         y_test[i] = 1
# y_test = y_test.astype(int)

In [44]:
# Prediction on test data with the help of trained model
y_pred = model.predict(X_test)

In [45]:
# Calculate performance metrics

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)
fpr, tpr, _ = roc_curve(y_test, y_pred)
precision, recall, _ = precision_recall_curve(y_test, y_pred)

In [46]:
# Print performance metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Confusion Matrix:\n", conf_matrix)
print("ROC AUC Score:", roc_auc)

Accuracy: 0.9408602150537635
Precision: [0.44086022 0.91764706 1.        ]
Recall: [1.         0.95121951 0.        ]
F1-score: 0.9341317365269461
Confusion Matrix:
 [[194  14]
 [  8 156]]
ROC AUC Score: 0.9419559099437148
